In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 # 1. AG用のnamespace作成

In [11]:
%%bash
kubectl create namespace mssql-ag


namespace/mssql-ag created


 # 2. AG用のsecret作成

In [12]:
%%bash
kubectl create secret generic sql-secrets --from-literal=sapassword=$PASSWORD --from-literal=masterkeypassword=$PASSWORD -n mssql-ag


secret/sql-secrets created


 # 3. operator作成

In [13]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/1.operator.yaml


# apiVersion: v1
# kind: Namespace
# metadata: {name: mssql-ag}
---
apiVersion: v1
kind: ServiceAccount
metadata: {name: mssql-operator, namespace: mssql-ag}
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata: {name: mssql-operator-ag}
rules:
- apiGroups: ['']
  resources: [serviceaccounts, services]
  verbs: [create, get, update, delete]
- apiGroups: [batch]
  resources: [jobs]
  verbs: [create, get, update, delete]
- apiGroups: [rbac.authorization.k8s.io]
  resources: [roles, rolebindings]
  verbs: [create, get, update, delete]
- apiGroups: [apps]
  resources: [statefulsets]
  verbs: [create, delete, get, update]
- apiGroups: ['']
  resources: [configmaps, endpoints, secrets]
  verbs: [create, get, update, watch, delete]
- apiGroups: ['']
  resources: [pods]
  verbs: [get, list, update]
- apiGroups: [apiextensions.k8s.io]
  resources: [customresourcedefinitions]
  verbs: [create]
- apiGroups: [apiextensions.k8s.io]
  resourceNames: [sqlservers.mssql.microsoft.com

In [14]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/1.operator.yaml -n mssql-ag


serviceaccount/mssql-operator created
clusterrole.rbac.authorization.k8s.io/mssql-operator-ag unchanged
clusterrolebinding.rbac.authorization.k8s.io/mssql-operator-ag unchanged
deployment.apps/mssql-operator created


 # 4. pod作成

In [15]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/2.sqlserver.yaml


apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql1, type: sqlservr}
  name: mssql1
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGroups: [mssql-ag]
  instanceRootVolumeClaimTemplate:
    accessModes: [ReadWriteOnce]
    resources:
      requests: {storage: 5Gi}
    storageClass: default
  saPassword:
    secretKeyRef: {key: sapassword, name: sql-secrets}
  sqlServerContainer: {image: 'mcr.microsoft.com/mssql/server:2019-CTP3.2-ubuntu'}
---
apiVersion: v1
kind: Service
metadata: {name: mssql1, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {name: mssql1, type: sqlservr}
  type: LoadBalancer
---
apiVersion: mssql.microsoft.com/v1
kind: SqlServer
metadata:
  labels: {name: mssql2, type: sqlservr}
  name: mssql2
  namespace: mssql-ag
spec:
  acceptEula: true
  agentsContainerImage: mcr.microsoft.com/mssql/ha:2019-CTP3.2-ubuntu
  availabilityGro

In [16]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/2.sqlserver.yaml -n mssql-ag


sqlserver.mssql.microsoft.com/mssql1 created
service/mssql1 created
sqlserver.mssql.microsoft.com/mssql2 created
service/mssql2 created
sqlserver.mssql.microsoft.com/mssql3 created
service/mssql3 created


 # 5. Service定義

In [17]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/AG/3.ag-services.yaml


apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-primary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433, targetPort: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: primary, type: sqlservr}
  type: LoadBalancer
---
apiVersion: v1
kind: Service
metadata: {annotations: null, name: ag-secondary, namespace: mssql-ag}
spec:
  ports:
  - {name: tds, port: 1433}
  selector: {role.ag.mssql.microsoft.com/mssql-ag: secondary,
    type: sqlservr}
  type: LoadBalancer
---

In [18]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/AG/3.ag-services.yaml -n mssql-ag


service/ag-primary created
service/ag-secondary created


 # 6. ag-primaryのExternalIPに接続

In [19]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'select @@servername'



--------------------------------------------------------------------------------------------------------------------------------
mssql3-0                                                                                                                        

(1 rows affected)


 # 7. AG用DB作成

In [21]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'create database agdb'
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'use agdb; create table tab01(id int, name varchar(max)); insert into tab01 values (1, "AG on K8s!")'
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'use agdb; select * from tab01'


Changed database context to 'agdb'.

(1 rows affected)
Changed database context to 'agdb'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 AG on K8s!                                                                                                                                                                                                                                                      

(1 rows affected)


 # 8. DBフルバックアップ

In [22]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'backup database agdb to disk="/home/o9o9/agdb.bak" with format'


Processed 336 pages for database 'agdb', file 'agdb' on file 1.
Processed 2 pages for database 'agdb', file 'agdb_log' on file 1.
BACKUP DATABASE successfully processed 338 pages in 0.421 seconds (6.262 MB/sec).


 # 9. AGへ登録

In [24]:
%%bash
sqlcmd -S 52.155.111.133 -U sa -P $PASSWORD -Q 'use master; ALTER AVAILABILITY GROUP [mssql-ag] ADD DATABASE agdb'


Changed database context to 'master'.
